# Experiments

In this notebook is implemented the three experiments described in the paper.

## Header

Libraries and additional programs

In [136]:
import matplotlib
import matplotlib.pyplot as plt
import numpy
import random
from scipy.stats.stats import pearsonr

%matplotlib inline
plt.rcParams["figure.figsize"] = [12,9]

pathh = '../Data/' 
#exp_file = 'ExperimentalResults_2.xlsx'  #Datos obtenidos con mis encuestas
#exp_file = 'ExperimentalResults_3.xlsx'   # Datos de MTURK-771
exp_file = '../Data/ExperimentalResults_4.xlsx'   # Q1 Modificado...

## Initializing memory and encoding dataset in HD space

In [137]:
%run EncodingDataset.ipynb
Init_mem()

End of memory initialization
Begining to encode dataset...


KeyboardInterrupt: 

## Experiment 1

Given a concept it retrieves the "num_concepts" most similar concepts to it (for each different method). A comparison between methods is done by counting the number of coincidences between lists of concepts.

In [ ]:
import random 

# List of all concepts in dataset
Concepts = get_concepts_list() 

# Test concepts
sample_size = 10
#Test_Concepts = random.sample(Concepts, sample_size)
Test_Concepts =  ['spoon'] #, 'airplane', 'chair']
print Test_Concepts

num_concepts = 11
coincidence = []

for test_concept in Test_Concepts:
    print "\n\nConcept: ", test_concept
    # Asking closest concept of another concept's definition...
    HDC_sim = HDvector.getLabelSP(Dict[test_concept].getPointer())[:num_concepts]
    HDC_sim = NormalizeHammDist(HDC_sim)
    print "HDC_sim: ", HDC_sim
    
    DatSet_sim = ClosestConcepts(test_concept, num_concepts)
    print "DatSet_sim: ", DatSet_sim

    concept = wn.synset( get_synset(test_concept) )
    # Path similarity
    LC_sim = apply_sim_metric(wn.lch_similarity, num_concepts, concept ) 
    print "LC_sim: ", LC_sim
    WUP_sim = apply_sim_metric(wn.wup_similarity, num_concepts, concept )
    print "WUP_sim: ", WUP_sim
    
    # Information Content
    Res_sim = apply_sim_metric(wn.res_similarity, num_concepts, concept, brown_ic)
    print "Res_sim: ", Res_sim
    JC_sim = apply_sim_metric(wn.jcn_similarity, num_concepts, concept, brown_ic)
    print "JC_sim: ", JC_sim
    
    # Comparing lists agains HDC_sim for counting coincidence.
    # Creating name sets
    HDC_names = set([x[0] for x in HDC_sim])
    DatSet_names = set([x[0] for x in DatSet_sim])
    LC_names = set([x[0] for x in LC_sim])
    WUP_names = set([x[0] for x in WUP_sim])
    Res_names = set([x[0] for x in Res_sim])
    JC_names = set([x[0] for x in JC_sim])
    
    coincidence = coincidence + [[len(HDC_names.intersection(DatSet_names)), 
                                 len(HDC_names.intersection(LC_names)), 
                                 len(HDC_names.intersection(WUP_names)),    
                                 len(HDC_names.intersection(Res_names)), 
                                 len(HDC_names.intersection(JC_names))]]
    print coincidence

coincidence = np.array(coincidence)
# Measuring average against each method.
print "Average: ", np.mean(coincidence, axis = 0)

## Experiment 2

### a) Human-judged similarity

In [ ]:
def ListofPairs (number):
    "It obtains a list of pairs of concepts"
    df = pd.read_excel(exp_file)
    if number > 0:
        string = 'Q' + str(number)    
    else:
        string = 'average'
    # 1) List of concepts
    ordered = df.sort_values(by=string)
    c1 = map(str, list( ordered['concept 1'] ))
    c2 = map(str, list( ordered['concept 2'] ))
    L1 = map(list, zip(c1,c2))
    
    # 2) Human similarity and stdev
    ordered = df.sort_values(by=string)
    L2 = list(ordered[string])
    L3 = list(ordered['stdev'])
    return L1, L2, L3

PConcepts, Hum_Sim, stdev = ListofPairs(1)


x = np.array(range(64))
stdev = np.array(map(lambda x: x*0.5, stdev))

plt.rcParams["figure.figsize"] = [11,7]

line,caps,bars = plt.errorbar(x, Hum_Sim, stdev,       
                    fmt="k-",
                    #linestyle='None',
                    marker='.',
                    markersize=5,
                    linewidth=0.3,
                    elinewidth=0.5,
                    ecolor='k',
                    capsize=3,
                    capthick=1.2)

# plt.setp(bars,label="Std dev")
# plt.legend(loc=('upper left'), prop={'size': 15})
plt.xlim((0,64))
plt.xlabel('Pair number', fontsize=18)
plt.ylabel('Semantic similarity', fontsize=18)
plt.grid(True)
plt.savefig('HumSim.png', dpi=300)
#plt.errorbar(x, Hum_Sim, stdev, linestyle='None', marker='.')

### b) Measuring similarity

In this cell we measure the similarity of each pair of concepts with each similarity measure.

In [ ]:
# Create dictionary of distances for each method
Dict_HD = {}; Dict_McRae = {}; Dict_wup = {}; Dict_lch = {}; Dict_res = {}; Dict_jcn = {}

# Loop over all pairs of concepts
key = 0
print "Starting"
for pair in PConcepts:
    print key,
    Dict_HD[key] = 1. - HDvector.dist(Dict[pair[0]].getPointer(), Dict[pair[1]].getPointer()) / float(N)
    Dict_McRae[key] = McRae_simi(pair)
    Dict_wup[key] = similarity_fun(wn.wup_similarity, pair)
    Dict_lch[key] = similarity_fun(wn.lch_similarity, pair)
    Dict_res[key] = similarity_fun(wn.res_similarity, pair, brown_ic)
    Dict_jcn[key] = similarity_fun(wn.jcn_similarity, pair, brown_ic)
    key += 1
print "End"

### c) Computing correlations

In [ ]:
# 1) Obtain "human list", it can be the average responses or an specific one
Hum_sim =  ListofPairs(3) #or Q + #0-30
    
# 2) Obtain list of ordered concepts (keys)
keys = map(lambda x: PConcepts.index(x), Hum_sim[0])
    
# 3) Create a list of distances according to each metric by consulting the appropiate dictionary
HD_sim = [Dict_HD[x] for x in keys]
McRae_sim = [Dict_McRae[x] for x in keys]
wup_sim = [Dict_wup[x] for x in keys]
lch_sim = [Dict_lch[x] for x in keys]
res_sim = [Dict_res[x] for x in keys]
jcn_sim = [Dict_jcn[x] for x in keys]

# 4) Calculate correlations
correlations = [['HDC', pearsonr(Hum_sim[1], HD_sim)[0]], ['McRae', pearsonr(Hum_sim[1], McRae_sim)[0]],
                ['wup', pearsonr(Hum_sim[1], wup_sim)[0]], ['lch', pearsonr(Hum_sim[1], lch_sim)[0]],
                ['res', pearsonr(Hum_sim[1], res_sim)[0]], ['jcn', pearsonr(Hum_sim[1], jcn_sim)[0]]]

print correlations

## d) Plotting

In [ ]:
# Plotting
plt.rcParams["figure.figsize"] = [16,12]
plt.subplots_adjust(hspace=0.4)
plt.subplot(321)
plt.plot(HD_sim, color='k', linestyle='None', marker='x', markersize=5, linewidth=1 )  

plt.title('HDC')
plt.xlabel('Pair number', fontsize=12)
plt.ylabel('Semantic similarity', fontsize=12)
plt.grid(True)

plt.subplot(322)
plt.plot(res_sim, color='k', linestyle='None', marker='.', markersize=5, linewidth=1 )  
plt.title('Resnik')
plt.xlabel('Pair number', fontsize=12)
plt.ylabel('Semantic similarity', fontsize=12)
plt.grid(True)

plt.subplot(323)
plt.plot(McRae_sim, color='k', linestyle='None', marker='^', markersize=5, linewidth=1 )  
plt.title('McRae')
plt.xlabel('Pair number', fontsize=12)
plt.ylabel('Semantic similarity', fontsize=12)
plt.grid(True)

plt.subplot(324)
plt.plot(jcn_sim, color='k', linestyle='None', marker='+', markersize=4, linewidth=1 )  
plt.title('Jiang & Conrath')
plt.xlabel('Pair number', fontsize=12)
plt.ylabel('Semantic similarity', fontsize=12)
plt.grid(True)

plt.subplot(325)
plt.plot(lch_sim, color='k', linestyle='None', marker='*', markersize=4, linewidth=1 )  
plt.title('Leacock & Chodorow')
plt.xlabel('Pair number', fontsize=12)
plt.ylabel('Semantic similarity', fontsize=12)
plt.grid(True)

plt.subplot(326)
plt.plot(wup_sim, color='k', linestyle='None', marker='.', markersize=5, linewidth=1 )  
plt.title('Wu & Palmer')
plt.xlabel('Pair number', fontsize=12)
plt.ylabel('Semantic similarity', fontsize=12)
plt.grid(True)


plt.savefig('SubPlots.png', dpi=300)